In [34]:
import numpy as np 
import pandas as pd 

from matminer.featurizers.site import LocalPropertyDifference
from matminer.featurizers.structure import SiteStatsFingerprint
from matminer.datasets.dataframe_loader import load_elastic_tensor
from matminer.featurizers.base import MultipleFeaturizer, BaseFeaturizer
from matminer.featurizers.stats import PropertyStats
from matminer.utils.data import MagpieData
from matminer.utils.caching import get_nearest_neighbors
from pymatgen.analysis.local_env import VoronoiNN

class VBS_mean(BaseFeaturizer):
    def __init__(self, properties, stats=('mean', 'std_dev'), deviation='AD'):
        self.properties = properties
        self.stats = tuple([stats]) if type(stats) == str else stats
        self.deviation = tuple([deviation]) if type(deviation) == str else deviation
        if self.stats and '_mode' in ''.join(self.stats):
            nmodes = 0
            for stat in self.stats:
                if '_mode' in stat and int(stat[0]) > nmodes:
                    nmodes = int(stat[0])
            self.nmodes = nmodes
            
#     @staticmethod
#     def from_preset(preset):
#         """
#         Create a new LocalPropertyDifference class according to a preset

#         Args:
#             preset (str) - Name of preset
#         """
#         if preset == "dejong2016":
#             return VBS_mean(
#                 properties=["Number", "AtomicWeight",
#                             "Column", "Row", "CovalentRadius",
#                             "Electronegativity"], 
#                 stats=['holder_mean::%d'%d for d in range(0, 4+1)] + ['geom_std_dev'],
#                 deviation='AD'
#                 ), 
#             VBS_mean(
#                 properties=["Number", "AtomicWeight",
#                             "Column", "Row", "CovalentRadius",
#                             "Electronegativity"], 
#                 stats=['holder_mean::%d'%d for d in [1, 2, 4]] + ['geom_std_dev'],
#                 deviation='SD'
#                 ) 
#         else:
#             raise ValueError('Unrecognized preset: ' + preset)
    
    def featurize(self, strc):
        
        ft = LocalPropertyDifference(properties=self.properties)
        _site_labels = ft.feature_labels()
        vals = []
        
        for dev in self.deviation:
            if dev == "AD":
                vals_AD = [[] for t in _site_labels]
                
                # Looping through the sites in structure and add featurized value of each site into vals
                for idx, site in enumerate(strc.sites):
                    opvalstmp = ft.featurize(strc, idx)
                    for j, opval in enumerate(opvalstmp):
                        if opval is None:
                            vals_AD[j].append(0.0)
                        else:
                            vals_AD[j].append(opval)
                vals += vals_AD

            elif dev == "SD":
                vals_SD = [[] for t in _site_labels]
                
                # Copied implementation from LocalPropertyDifference, removing only the np.abs portion
                # (how else to more efficiently implement this?)
                # Looping through the sites in structure and add featurized value of each site into vals
                for idx, site in enumerate(strc.sites):
                    # Get the targeted site
                    my_site = strc[idx]

                    # Get the tessellation of a site
                    nn = get_nearest_neighbors(VoronoiNN(weight='area'), strc, idx)

                    # Get the element and weight of each site
                    elems = [n['site'].specie for n in nn]
                    weights = [n['weight'] for n in nn]

                    # Compute the difference for each property
                    opvalstmp = np.zeros((len(self.properties),))
                    total_weight = np.sum(weights)
                    for i,p in enumerate(self.properties):
                        my_prop = MagpieData().get_elemental_property(my_site.specie, p)
                        n_props = MagpieData().get_elemental_properties(elems, p)
                        opvalstmp[i] = np.dot(weights, np.subtract(n_props, my_prop)) / total_weight
                        
                    for j, opval in enumerate(opvalstmp):
                        if opval is None:
                            vals_SD[j].append(0.0)
                        else:
                            vals_SD[j].append(opval)
                vals += vals_SD
            else:
                raise ValueError('Unrecognized mean difference type ' + dev)
                
        # calculate stats using the featurized values        
        if self.stats:
            stats = []
            for op in vals:
                for stat in self.stats:
                    stats.append(PropertyStats().calc_stat(op, stat))

            return stats
        else:
            return vals
    
    def feature_labels(self):
        if self.stats and self.deviation:
            labels = []
            for dev in self.deviation:
                for prop in self.properties:
                    for stat in self.stats:
                        labels.append("%s %s %s" %(dev, stat, prop))
            return labels
        else:
            return LocalPropertyDifference(properties=self.properties).feature_labels()

Questions:
- How to make the from_preset method for dejong? 
- How to speed up SD portion? (implementation copied from LocalPropertyDifference takes up more time)
- PropertyStats.HolderMeans seems to differ from the formula used in deJong's paper (because of the weights). Is it okay to use them like that? And how to input the weights in the function parameter??
- Dividing by zero and log of zero encountered a lot in std_dev. 

In [35]:
from matminer.datasets.dataframe_loader import load_elastic_tensor

data = load_elastic_tensor()
c = data['structure'][0]

In [36]:
#AD
ft = VBS_mean(properties=["Number", "AtomicWeight",
                          "Column", "Row", "CovalentRadius",
                          "Electronegativity"], 
              stats=['holder_mean::%d'%d for d in range(0, 4+1)] + ['std_dev'],
              deviation='AD')
d = ft.featurize_dataframe(data, col_id='structure')
d

/Users/AikRui/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning:

divide by zero encountered in log

/Users/AikRui/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning:

divide by zero encountered in log

/Users/AikRui/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning:

divide by zero encountered in log

/Users/AikRui/anaconda3/lib/python3.6/site-packages/scipy/stats/stats.py:305: RuntimeWarning:

divide by zero encountered in log



,material_id,formula,nsites,space_group,volume,structure,elastic_anisotropy,G_Reuss,G_VRH,G_Voigt,...,AD holder_mean::2 CovalentRadius,AD holder_mean::3 CovalentRadius,AD holder_mean::4 CovalentRadius,AD std_dev CovalentRadius,AD holder_mean::0 Electronegativity,AD holder_mean::1 Electronegativity,AD holder_mean::2 Electronegativity,AD holder_mean::3 Electronegativity,AD holder_mean::4 Electronegativity,AD std_dev Electronegativity
0,mp-10003,Nb4CoSi,12,124,194.419802,"[[0.94814328 2.07280467 2.5112 ] Nb, [5.273...",0.030688,96.844535,97.141604,97.438674,...,25.558050,27.779116,29.755384,1.040783e+01,0.137083,0.148743,0.161669,0.173943,0.184335,6.334563e-02
1,mp-10010,Al(CoSi)2,5,164,61.987320,"[[0. 0. 0.] Al, [1.96639263 1.13529553 0.75278...",0.266910,93.939650,96.252006,98.564362,...,9.231011,9.287459,9.337628,1.078476e+00,0.104610,0.122193,0.145556,0.168810,0.187807,7.909063e-02
2,mp-10015,SiOs,2,221,25.952539,"[[1.480346 1.480346 1.480346] Si, [0. 0. 0.] Os]",0.756489,120.962289,130.112955,139.263621,...,25.607695,25.607695,25.607695,0.000000e+00,0.232797,0.232797,0.232797,0.232797,0.232797,0.000000e+00
3,mp-10021,Ga,4,63,76.721433,"[[0. 1.09045794 0.84078375] Ga, [0. ...",2.376805,12.205989,15.101901,17.997812,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
4,mp-10025,SiRu2,12,62,160.300999,"[[1.0094265 4.24771709 2.9955487 ] Si, [3.028...",0.196930,100.110773,101.947798,103.784823,...,23.775608,25.160206,26.388127,8.098499e+00,0.180445,0.191604,0.203791,0.215659,0.226184,6.941571e-02
5,mp-10037,AlCo3C,5,221,51.574959,"[[0. 0. 0.] Al, [0. 1.861157 1.861157] C...",0.420936,111.795761,116.501644,121.207527,...,25.265282,30.004681,33.634248,1.550996e+01,0.327224,0.352846,0.386847,0.424569,0.460024,1.585874e-01
6,mp-10063,CdSnSb2,16,122,580.176940,"[[0. 3.322252 9.855906] Cd, [0. 0....",0.629264,16.692188,17.742410,18.792631,...,2.790220,3.131199,3.394239,1.498747e+00,0.186058,0.201846,0.217515,0.232001,0.244698,8.105938e-02
7,mp-101,Ir,4,225,58.258386,"[[0. 0. 0.] Ir, [0. 1.938308 1.938308] I...",0.174540,212.791803,216.505869,220.219935,...,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
8,mp-10125,SbIr,4,194,80.054967,"[[2.03154403 1.17291049 4.19959275] Sb, [-2.03...",0.591712,55.983343,59.245410,62.507476,...,1.627537,1.629387,1.631226,7.770582e-02,0.121787,0.121926,0.122065,0.122204,0.122342,5.827936e-03
9,mp-10154,MnSbIr,12,216,228.873769,"[[3.0584545 0. 0. ] Mn, [3.058454...",0.109313,39.747198,40.181671,40.616145,...,1.348102,1.454819,1.544111,5.411267e-01,0.429349,0.438263,0.447627,0.457099,0.466334,9.107836e-02


In [38]:
#SD
ft = VBS_mean(properties=["Number", "AtomicWeight",
                            "Column", "Row", "CovalentRadius",
                            "Electronegativity"], 
                stats=['holder_mean::%d'%d for d in [1, 2, 4]] + ['std_dev'],
                deviation='SD') 
d = ft.featurize_dataframe(data, col_id='structure')
d

Process ForkPoolWorker-11:
Process ForkPoolWorker-9:
Process ForkPoolWorker-10:


KeyboardInterrupt: 